# Amazon Product Length Prediction

**Approach**: Multi-embedding ensemble with KNN retrieval features  
**Architecture**: Pre-computed embeddings → MLP regressor  
**Metric**: MAPE (Mean Absolute Percentage Error)

## 1. Setup

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from kaggle_secrets import UserSecretsClient
from pytorch_lightning.loggers import WandbLogger
from dataclasses import dataclass, field
from typing import Optional
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime

# Weights & Biases for experiment tracking
import wandb

# Reproducibility
SEED = 42
pl.seed_everything(SEED)

print(f"PyTorch: {torch.__version__}")
print(f"Lightning: {pl.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Seed set to 42


PyTorch: 2.8.0+cu126
Lightning: 2.6.0
CUDA available: True
GPU: Tesla P100-PCIE-16GB


In [2]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")

try:
    wandb.login(key=secret_value_0)
except Exception as e:
    wandb.login()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bhanu-prasanna2001 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 2. Configuration

## 2.1 Quick Experiments (Change Config Here)

**Recommended experiment order:**
1. `loss_fn="mape"` - Baseline (what you have: ~52%)
2. `loss_fn="rmsle"` - Better gradient flow, penalizes under-prediction
3. `loss_fn="log_mape"` - Handles wide value range
4. `loss_fn="focal_mape", focal_gamma=2.0` - Focus on hard examples
5. `loss_fn="combined"` - Balance MAPE + RMSLE
6. `use_log_target=True` with any loss - Compress target range

In [3]:
@dataclass
class Config:
    """Training configuration."""
    # Paths (adjust for Kaggle)
    data_dir: Path = Path("/kaggle/input/amazon-ml-challenge-2023/total_sentence_data/total_sentence_data")
    output_dir: Path = Path("/kaggle/working")
    embk20_dir: Path = Path("/kaggle/input/product-length-embeddings")
    
    # Data splits
    train_ratio: float = 0.85
    val_ratio: float = 0.10
    test_ratio: float = 0.05
    
    # Embeddings to use
    active_embeddings: list = field(default_factory=lambda: ["minilm", "mpnet", "distiluse", "e5small"])
    embedding_dims: dict = field(default_factory=lambda: {
        "minilm": 384, "mpnet": 768, "distiluse": 512, "e5small": 384
    })
    
    # KNN features
    use_knn: bool = True
    knn_k: int = 20
    knn_embeddings: list = field(default_factory=lambda: ["minilm", "mpnet", "distiluse", "e5small"])
    knn_ensemble: str = "concat"  # "concat" or "mean"
    
    # Model architecture
    product_type_emb_dim: int = 128
    hidden_dims: list = field(default_factory=lambda: [1024, 256, 64])
    dropout: float = 0.2
    use_batch_norm: bool = True
    
    # Training
    batch_size: int = 512
    lr: float = 1e-3
    weight_decay: float = 0.01
    epochs: int = 30
    warmup_ratio: float = 0.05
    patience: int = 5
    gradient_clip_val: float = 1.0
    num_workers: int = 2
    
    # Loss function: "mape", "weighted_mape", "smape", "log_mape", "rmsle", "huber", "mse", "focal_mape", "combined"
    loss_fn: str = "mape"
    huber_delta: float = 1.0
    focal_gamma: float = 2.0  # For focal_mape
    combined_alpha: float = 0.7  # For combined loss: alpha * MAPE + (1-alpha) * RMSLE
    
    # Log-transform target (helps with skewed distributions)
    use_log_target: bool = True
    
    # W&B Logging
    wandb_project: str = "amazon-product-length"
    wandb_run_name: Optional[str] = None
    log_every_n_steps: int = 50
    val_check_interval: float = 0.25
    
    @property
    def total_embedding_dim(self) -> int:
        return sum(self.embedding_dims[e] for e in self.active_embeddings)
    
    @property
    def knn_dim(self) -> int:
        if not self.use_knn:
            return 0
        n_features = 5  # mean, std, median, min, max
        if self.knn_ensemble == "concat":
            return n_features * len(self.knn_embeddings)
        return n_features
    
    def to_dict(self) -> dict:
        """Convert config to dict for W&B logging."""
        return {
            "data_dir": str(self.data_dir),
            "output_dir": str(self.output_dir),
            "train_ratio": self.train_ratio,
            "val_ratio": self.val_ratio,
            "test_ratio": self.test_ratio,
            "active_embeddings": self.active_embeddings,
            "total_embedding_dim": self.total_embedding_dim,
            "use_knn": self.use_knn,
            "knn_k": self.knn_k,
            "knn_embeddings": self.knn_embeddings,
            "knn_ensemble": self.knn_ensemble,
            "knn_dim": self.knn_dim,
            "product_type_emb_dim": self.product_type_emb_dim,
            "hidden_dims": self.hidden_dims,
            "dropout": self.dropout,
            "use_batch_norm": self.use_batch_norm,
            "batch_size": self.batch_size,
            "lr": self.lr,
            "weight_decay": self.weight_decay,
            "epochs": self.epochs,
            "warmup_ratio": self.warmup_ratio,
            "patience": self.patience,
            "gradient_clip_val": self.gradient_clip_val,
            "loss_fn": self.loss_fn,
            "huber_delta": self.huber_delta,
            "focal_gamma": self.focal_gamma,
            "combined_alpha": self.combined_alpha,
            "use_log_target": self.use_log_target,
        }


cfg = Config()
print(f"Total embedding dim: {cfg.total_embedding_dim}")
print(f"KNN dim: {cfg.knn_dim}")
print(f"Active embeddings: {cfg.active_embeddings}")
print(f"Loss function: {cfg.loss_fn}")
print(f"Log-target: {cfg.use_log_target}")

Total embedding dim: 2048
KNN dim: 20
Active embeddings: ['minilm', 'mpnet', 'distiluse', 'e5small']
Loss function: mape
Log-target: True


In [4]:
# =============================================================================
# 🔧 EXPERIMENT CONFIG - MODIFY THIS CELL TO TEST DIFFERENT SETTINGS
# =============================================================================

# Quick experiment toggles
EXPERIMENT = {
    "loss_fn": "mape",           # Try: "mape", "rmsle", "log_mape", "focal_mape", "smape", "combined"
    "use_log_target": True,      # True compresses the target range (1-5000) → (0-8.5)
    "focal_gamma": 2.0,          # Only used if loss_fn="focal_mape"
    "huber_delta": 1.0,          # Only used if loss_fn="huber"
    "combined_alpha": 0.7,       # Only used if loss_fn="combined" (alpha * MAPE + (1-alpha) * RMSLE)
    "wandb_run_name": None,      # Set to custom name or None for auto-generated
}

# Override config with experiment settings
cfg.loss_fn = EXPERIMENT["loss_fn"]
cfg.use_log_target = EXPERIMENT["use_log_target"]
cfg.focal_gamma = EXPERIMENT["focal_gamma"]
cfg.huber_delta = EXPERIMENT["huber_delta"]
cfg.combined_alpha = EXPERIMENT["combined_alpha"]
cfg.wandb_run_name = EXPERIMENT["wandb_run_name"]

print("=" * 60)
print(f"🧪 EXPERIMENT: loss_fn={cfg.loss_fn}, log_target={cfg.use_log_target}")
print("=" * 60)

🧪 EXPERIMENT: loss_fn=mape, log_target=True


## 2.2 W&B Logger Setup

In [5]:
# =============================================================================
# W&B Logger and Custom Callbacks
# =============================================================================

def create_wandb_logger(config: Config) -> WandbLogger:
    """Create W&B logger for experiment tracking."""
    run_name = config.wandb_run_name or f"ensemble_{config.loss_fn}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    return WandbLogger(
        project=config.wandb_project,
        name=run_name,
        config=config.to_dict(),
        save_dir=str(config.output_dir),
    )


class SamplePredictionCallback(pl.Callback):
    """Logs sample predictions to W&B for visual inspection."""
    
    def __init__(self, num_samples: int = 100):
        self.num_samples = num_samples
        
    def on_validation_epoch_end(self, trainer, pl_module):
        if not trainer.val_dataloaders:
            return
            
        preds, targets, types = [], [], []
        pl_module.eval()
        device = pl_module.device
        
        with torch.no_grad():
            for batch in trainer.val_dataloaders:
                knn = batch.get("knn_features")
                pred = pl_module(
                    batch["text_embedding"].to(device),
                    batch["product_type"].to(device),
                    knn_features=knn.to(device) if knn is not None else None,
                )
                preds.append(pred.cpu().numpy())
                targets.append(batch["target"].numpy())
                types.append(batch["product_type"].numpy())
                
                if sum(len(p) for p in preds) >= self.num_samples * 2:
                    break
        
        preds, targets, types = np.concatenate(preds), np.concatenate(targets), np.concatenate(types)
        idx = np.random.choice(len(preds), min(self.num_samples, len(preds)), replace=False)
        
        # Handle log-target transform if applicable
        if pl_module.use_log_target:
            targets = np.expm1(targets)
            preds = np.expm1(preds)
        
        preds = np.maximum(preds, EPSILON)
        
        data = [
            [targets[i], preds[i], abs(targets[i] - preds[i]), abs(targets[i] - preds[i]) / targets[i] * 100, int(types[i])]
            for i in idx
        ]
        table = wandb.Table(columns=["true", "pred", "abs_error", "pct_error", "product_type"], data=data)
        wandb.log({
            "val_samples": table,
            "epoch": trainer.current_epoch,
            "pred_vs_true": wandb.plot.scatter(table, "true", "pred", title=f"Epoch {trainer.current_epoch}")
        })


class MetricHistoryCallback(pl.Callback):
    """Tracks MAPE history across epochs."""
    
    def __init__(self):
        self.train_mape: list[float] = []
        self.val_mape: list[float] = []
        
    def on_train_epoch_end(self, trainer, pl_module):
        if "train_mape" in trainer.callback_metrics:
            self.train_mape.append(trainer.callback_metrics["train_mape"].item())
            
    def on_validation_epoch_end(self, trainer, pl_module):
        if "val_mape" in trainer.callback_metrics:
            self.val_mape.append(trainer.callback_metrics["val_mape"].item())


print("W&B logger and callbacks defined ✓")

W&B logger and callbacks defined ✓


## 3. Constants & Utilities

In [6]:
EPSILON = 1e-6
SCORE_SCALE = 100.0

# =============================================================================
# Loss Functions Registry
# =============================================================================

def mape_loss(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """MAPE loss - directly optimizes competition metric."""
    return torch.mean(torch.abs(target - pred) / target)


def weighted_mape_loss(pred: torch.Tensor, target: torch.Tensor, scale: float = 100.0, min_weight: float = 0.1) -> torch.Tensor:
    """Weighted MAPE - downweights small targets to reduce their outsized influence."""
    weights = torch.clamp(target / scale, min=min_weight, max=1.0)
    return torch.mean(weights * torch.abs(target - pred) / target)


def smape_loss(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Symmetric MAPE - bounded [0, 2], handles zeros gracefully."""
    numerator = torch.abs(pred - target)
    denominator = torch.abs(pred) + torch.abs(target) + EPSILON
    return torch.mean(2.0 * numerator / denominator)


def log_mape_loss(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Log-MAPE - better for wide value ranges (1 to 5000)."""
    log_pred = torch.log(pred + EPSILON)
    log_target = torch.log(target + EPSILON)
    return torch.mean(torch.abs(log_pred - log_target) / (torch.abs(log_target) + EPSILON))


def rmsle_loss(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """RMSLE - penalizes under-prediction more than over-prediction."""
    return torch.sqrt(torch.mean((torch.log1p(pred) - torch.log1p(target)) ** 2))


def huber_loss(pred: torch.Tensor, target: torch.Tensor, delta: float = 1.0) -> torch.Tensor:
    """Huber loss in log-space - robust to outliers."""
    return F.huber_loss(torch.log1p(pred), torch.log1p(target), delta=delta)


def mse_loss(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """MSE in log-space - simple baseline."""
    return F.mse_loss(torch.log1p(pred), torch.log1p(target))


def focal_mape_loss(pred: torch.Tensor, target: torch.Tensor, gamma: float = 2.0) -> torch.Tensor:
    """Focal MAPE - focuses learning on hard examples with high error."""
    ape = torch.abs(target - pred) / target  # Absolute percentage error
    focal_weight = (ape ** gamma)  # Hard examples get higher weight
    focal_weight = focal_weight / (focal_weight.mean() + EPSILON)  # Normalize
    return torch.mean(focal_weight * ape)


def combined_loss(pred: torch.Tensor, target: torch.Tensor, alpha: float = 0.7) -> torch.Tensor:
    """Combined MAPE + RMSLE - balances percentage error with log-scale penalty."""
    return alpha * mape_loss(pred, target) + (1 - alpha) * rmsle_loss(pred, target)


# Loss registry
LOSS_REGISTRY = {
    "mape": mape_loss,
    "weighted_mape": weighted_mape_loss,
    "smape": smape_loss,
    "log_mape": log_mape_loss,
    "rmsle": rmsle_loss,
    "huber": huber_loss,
    "mse": mse_loss,
    "focal_mape": focal_mape_loss,
    "combined": combined_loss,
}


def get_loss_fn(name: str, **kwargs):
    """Get loss function by name with optional kwargs."""
    if name not in LOSS_REGISTRY:
        raise ValueError(f"Unknown loss: {name}. Available: {list(LOSS_REGISTRY.keys())}")
    fn = LOSS_REGISTRY[name]
    if kwargs:
        return lambda p, t: fn(p, t, **kwargs)
    return fn


# =============================================================================
# Metrics (PyTorch)
# =============================================================================

def compute_mape(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """MAPE as percentage for logging (0-100+)."""
    with torch.no_grad():
        pred_safe = F.relu(pred) + EPSILON
        return torch.mean(torch.abs(target - pred_safe) / target) * SCORE_SCALE


def compute_rmsle(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Root Mean Squared Logarithmic Error."""
    with torch.no_grad():
        pred_safe = F.relu(pred) + EPSILON
        return torch.sqrt(torch.mean((torch.log1p(pred_safe) - torch.log1p(target)) ** 2))


def compute_score(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Competition score: max(0, 100 * (1 - MAPE/100))."""
    return torch.clamp(SCORE_SCALE - compute_mape(pred, target), min=0)


# =============================================================================
# Metrics (NumPy)
# =============================================================================

def mape_numpy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """MAPE as percentage (NumPy)."""
    y_pred_safe = np.maximum(y_pred, EPSILON)
    return float(np.mean(np.abs((y_true - y_pred_safe) / y_true)) * SCORE_SCALE)


def rmsle_numpy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """RMSLE (NumPy)."""
    y_pred_safe = np.maximum(y_pred, EPSILON)
    return float(np.sqrt(np.mean((np.log1p(y_pred_safe) - np.log1p(y_true)) ** 2)))


def score_numpy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Competition score (NumPy)."""
    return max(0.0, SCORE_SCALE * (1 - mape_numpy(y_true, y_pred) / SCORE_SCALE))


print(f"Available losses: {list(LOSS_REGISTRY.keys())}")

Available losses: ['mape', 'weighted_mape', 'smape', 'log_mape', 'rmsle', 'huber', 'mse', 'focal_mape', 'combined']


## 4. Post-Processing (Snapping)

In [7]:
def snap_to_nearest(preds: np.ndarray, valid_lengths: np.ndarray) -> np.ndarray:
    """Snap predictions to nearest valid length using binary search."""
    if len(valid_lengths) == 0:
        return preds
    valid_sorted = np.sort(valid_lengths)
    indices = np.searchsorted(valid_sorted, preds)
    left_idx = np.maximum(indices - 1, 0)
    right_idx = np.minimum(indices, len(valid_sorted) - 1)
    left_vals, right_vals = valid_sorted[left_idx], valid_sorted[right_idx]
    return np.where(np.abs(preds - left_vals) <= np.abs(preds - right_vals), left_vals, right_vals)


@dataclass
class Snapper:
    """Snaps predictions to valid product lengths seen in training data."""
    all_valid_lengths: np.ndarray = None
    lengths_by_type: dict = field(default_factory=dict)
    min_length: float = 1.0
    max_length: float = 5000.0
    
    def snap_by_type(self, preds: np.ndarray, product_types: np.ndarray) -> np.ndarray:
        """Snap using type-specific valid lengths."""
        if not self.lengths_by_type:
            return snap_to_nearest(preds, self.all_valid_lengths)
        result = preds.copy()
        for ptype in np.unique(product_types):
            mask = product_types == ptype
            type_lengths = self.lengths_by_type.get(int(ptype), self.all_valid_lengths)
            if type_lengths is not None and len(type_lengths) > 0:
                result[mask] = snap_to_nearest(preds[mask], type_lengths)
        return result
    
    def process(self, preds: np.ndarray, product_types: np.ndarray) -> np.ndarray:
        """Full post-processing: ensure positive → snap → clip."""
        result = np.maximum(preds, EPSILON)
        result = self.snap_by_type(result, product_types)
        return np.clip(result, self.min_length, self.max_length)


def create_snapper(train_targets: np.ndarray, train_product_types: np.ndarray) -> Snapper:
    """Create Snapper from training data."""
    all_valid = np.sort(np.unique(train_targets))
    lengths_by_type = {
        int(pt): np.sort(np.unique(train_targets[train_product_types == pt]))
        for pt in np.unique(train_product_types)
    }
    print(f"Snapper: {len(all_valid):,} unique lengths, {len(lengths_by_type):,} product types")
    return Snapper(
        all_valid_lengths=all_valid,
        lengths_by_type=lengths_by_type,
        min_length=float(train_targets.min()),
        max_length=float(train_targets.max()),
    )

## 5. Dataset

In [8]:
class EmbeddingDataset(Dataset):
    """Dataset for pre-computed embeddings with optional KNN features."""
    
    def __init__(
        self,
        embeddings: dict[str, np.ndarray],
        indices: np.ndarray,
        product_types: np.ndarray,
        targets: Optional[np.ndarray] = None,
        product_ids: Optional[np.ndarray] = None,
        knn_features: Optional[dict[str, np.ndarray]] = None,
        knn_ensemble: str = "concat",
        use_log_target: bool = False,
    ):
        self.embeddings = embeddings
        self.indices = indices
        self.product_types = product_types
        self.targets = targets
        self.product_ids = product_ids
        self.knn_features = knn_features
        self.knn_ensemble = knn_ensemble
        self.use_log_target = use_log_target
        self.model_names = list(embeddings.keys())
        
    def __len__(self) -> int:
        return len(self.indices)
    
    def __getitem__(self, idx: int) -> dict:
        real_idx = self.indices[idx]
        
        # Concatenate embeddings
        emb_list = [self.embeddings[name][real_idx] for name in self.model_names]
        text_embedding = np.concatenate(emb_list, axis=0)
        
        item = {
            "text_embedding": torch.tensor(text_embedding, dtype=torch.float32),
            "product_type": torch.tensor(self.product_types[idx], dtype=torch.long),
        }
        
        if self.targets is not None:
            target = self.targets[idx]
            if self.use_log_target:
                target = np.log1p(target)  # log(1 + x) transform
            item["target"] = torch.tensor(target, dtype=torch.float32)
            
        if self.product_ids is not None:
            item["product_id"] = self.product_ids[idx]
        
        if self.knn_features is not None:
            knn_list = [self.knn_features[name][real_idx] for name in sorted(self.knn_features.keys())]
            if self.knn_ensemble == "mean":
                combined = np.mean(knn_list, axis=0)
            else:
                combined = np.concatenate(knn_list, axis=0)
            item["knn_features"] = torch.tensor(combined, dtype=torch.float32)
            
        return item

## 6. Model

In [9]:
class MLPHead(nn.Module):
    """MLP regressor with batch normalization and dropout."""
    
    def __init__(self, input_dim: int, hidden_dims: list, dropout: float = 0.2, use_batch_norm: bool = True):
        super().__init__()
        layers = []
        dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(dim, h))
            if use_batch_norm:
                layers.append(nn.BatchNorm1d(h))
            layers.extend([nn.ReLU(), nn.Dropout(dropout)])
            dim = h
        layers.append(nn.Linear(dim, 1))
        self.net = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x).squeeze(-1)


class EnsembleModel(pl.LightningModule):
    """Embedding ensemble → MLP regressor for product length prediction.
    
    Combines pre-computed text embeddings with product type embeddings and
    optional KNN features, then feeds through MLP to predict length.
    Supports log-target transform for handling skewed distributions.
    """
    
    def __init__(self, config: Config, num_product_types: int):
        super().__init__()
        self.save_hyperparameters(ignore=["config"])
        self.config = config
        self.use_log_target = config.use_log_target
        
        # Product type embedding (index 0 reserved for unknown)
        self.product_emb = nn.Embedding(num_product_types, config.product_type_emb_dim, padding_idx=0)
        nn.init.normal_(self.product_emb.weight, mean=0, std=0.02)
        
        # Input dimension calculation
        input_dim = config.total_embedding_dim + config.product_type_emb_dim
        
        # Optional KNN feature projection (32-dim learned projection)
        if config.knn_dim > 0:
            self.knn_proj = nn.Sequential(nn.Linear(config.knn_dim, 32), nn.ReLU(), nn.Linear(32, 32))
            input_dim += 32
        else:
            self.knn_proj = None
        
        self.head = MLPHead(input_dim, config.hidden_dims, config.dropout, config.use_batch_norm)
        
        # Configure loss function with appropriate kwargs
        loss_kwargs = {}
        if config.loss_fn == "huber":
            loss_kwargs["delta"] = config.huber_delta
        elif config.loss_fn == "focal_mape":
            loss_kwargs["gamma"] = config.focal_gamma
        elif config.loss_fn == "combined":
            loss_kwargs["alpha"] = config.combined_alpha
        self.loss_fn = get_loss_fn(config.loss_fn, **loss_kwargs)
        
    def forward(self, text_embedding: torch.Tensor, product_type: torch.Tensor, knn_features: Optional[torch.Tensor] = None) -> torch.Tensor:
        type_emb = self.product_emb(product_type)
        
        if knn_features is not None and self.knn_proj is not None:
            parts = [text_embedding, type_emb, self.knn_proj(knn_features)]
        else:
            parts = [text_embedding, type_emb]
            
        return self.head(torch.cat(parts, dim=-1))
    
    def _compute_metrics(self, pred: torch.Tensor, target: torch.Tensor) -> dict[str, torch.Tensor]:
        """Compute all metrics for logging."""
        with torch.no_grad():
            pred_safe = F.relu(pred) + EPSILON
            return {
                "mape": compute_mape(pred_safe, target),
                "rmsle": compute_rmsle(pred_safe, target),
                "score": compute_score(pred_safe, target),
            }
    
    def _step(self, batch: dict, stage: str) -> torch.Tensor:
        pred = self(batch["text_embedding"], batch["product_type"], batch.get("knn_features"))
        pred_safe = F.relu(pred) + EPSILON
        
        target = batch["target"]
        if self.use_log_target:
            # Transform back for loss/metrics computation
            target_linear = torch.expm1(target)
            pred_linear = torch.expm1(pred_safe)
            loss = self.loss_fn(pred_linear, target_linear)
            metrics = self._compute_metrics(pred_linear, target_linear)
        else:
            loss = self.loss_fn(pred_safe, target)
            metrics = self._compute_metrics(pred_safe, target)
        
        # Log all metrics
        self.log(f"{stage}_loss", loss, prog_bar=True, on_epoch=True)
        self.log(f"{stage}_mape", metrics["mape"], prog_bar=True, on_epoch=True)
        self.log(f"{stage}_rmsle", metrics["rmsle"], on_epoch=True)
        self.log(f"{stage}_score", metrics["score"], on_epoch=True)
        return loss
    
    def training_step(self, batch: dict, batch_idx: int) -> torch.Tensor:
        return self._step(batch, "train")
    
    def validation_step(self, batch: dict, batch_idx: int) -> torch.Tensor:
        return self._step(batch, "val")
    
    def test_step(self, batch: dict, batch_idx: int) -> torch.Tensor:
        return self._step(batch, "test")
    
    def predict_step(self, batch: dict, batch_idx: int) -> torch.Tensor:
        pred = self(batch["text_embedding"], batch["product_type"], batch.get("knn_features"))
        pred_safe = F.relu(pred) + EPSILON
        if self.use_log_target:
            pred_safe = torch.expm1(pred_safe)
        return pred_safe
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.config.lr, weight_decay=self.config.weight_decay)
        total_steps = int(self.trainer.estimated_stepping_batches)
        warmup_steps = int(self.config.warmup_ratio * total_steps)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer, max_lr=self.config.lr, total_steps=total_steps,
            pct_start=warmup_steps / total_steps, anneal_strategy="cos"
        )
        return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "interval": "step"}}

## 7. Data Loading

In [10]:
def load_data(cfg: Config):
    """Load embeddings, KNN features, and metadata."""
    print("Loading data...")
    
    # Load train CSV for targets and product types
    train_csv = cfg.data_dir / "total_sentence_train.csv"
    test_csv = cfg.data_dir / "total_sentence_test.csv"
    
    train_df = pd.read_csv(train_csv)
    test_df = pd.read_csv(test_csv)
    
    print(f"Train samples: {len(train_df):,}")
    print(f"Test samples: {len(test_df):,}")
    
    # Load embeddings (memory-mapped for efficiency)
    embeddings = {}
    for emb_name in cfg.active_embeddings:
        path = cfg.embk20_dir / f"{emb_name}_train.npy"
        embeddings[emb_name] = np.load(path, mmap_mode="r")
        print(f"Loaded {emb_name}: {embeddings[emb_name].shape}")
    
    # Load KNN features
    knn_features = None
    if cfg.use_knn:
        knn_features = {}
        for emb_name in cfg.knn_embeddings:
            path = cfg.embk20_dir / f"knn_k{cfg.knn_k}_{emb_name}_train.npy"
            knn_features[emb_name] = np.load(path, mmap_mode="r")
            print(f"Loaded KNN ({emb_name}): {knn_features[emb_name].shape}")
    
    # Build product type mapping
    all_types = pd.concat([train_df["PRODUCT_TYPE_ID"], test_df["PRODUCT_TYPE_ID"]]).unique()
    type_to_idx = {t: i + 1 for i, t in enumerate(sorted(all_types))}  # 0 reserved for unknown
    num_product_types = len(type_to_idx) + 1
    print(f"Product types: {num_product_types:,}")
    
    # Extract arrays
    targets = train_df["PRODUCT_LENGTH"].values.astype(np.float32)
    product_types = train_df["PRODUCT_TYPE_ID"].map(type_to_idx).values
    
    return {
        "embeddings": embeddings,
        "knn_features": knn_features,
        "targets": targets,
        "product_types": product_types,
        "num_product_types": num_product_types,
        "type_to_idx": type_to_idx,
        "train_df": train_df,
        "test_df": test_df,
    }

In [11]:
def create_splits(cfg: Config, data: dict):
    """Create train/val/test splits."""
    n = len(data["targets"])
    indices = np.random.permutation(n)
    
    train_end = int(cfg.train_ratio * n)
    val_end = train_end + int(cfg.val_ratio * n)
    
    splits = {
        "train": indices[:train_end],
        "val": indices[train_end:val_end],
        "test": indices[val_end:],
    }
    
    print(f"Splits: train={len(splits['train']):,}, val={len(splits['val']):,}, test={len(splits['test']):,}")
    return splits

In [12]:
def create_dataloaders(cfg: Config, data: dict, splits: dict):
    """Create DataLoaders for each split."""
    
    def make_dataset(split_indices: np.ndarray, include_targets: bool = True):
        return EmbeddingDataset(
            embeddings=data["embeddings"],
            indices=split_indices,
            product_types=data["product_types"][split_indices],
            targets=data["targets"][split_indices] if include_targets else None,
            knn_features=data["knn_features"],
            knn_ensemble=cfg.knn_ensemble,
            use_log_target=cfg.use_log_target,
        )
    
    train_ds = make_dataset(splits["train"])
    val_ds = make_dataset(splits["val"])
    test_ds = make_dataset(splits["test"])
    
    train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=cfg.batch_size * 2, shuffle=False, num_workers=cfg.num_workers, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=cfg.batch_size * 2, shuffle=False, num_workers=cfg.num_workers, pin_memory=True)
    
    return train_loader, val_loader, test_loader, {"train": train_ds, "val": val_ds, "test": test_ds}

## 8. Training

In [13]:
# Load data
data = load_data(cfg)
splits = create_splits(cfg, data)
train_loader, val_loader, test_loader, datasets = create_dataloaders(cfg, data, splits)

Loading data...
Train samples: 2,173,199
Test samples: 734,736
Loaded minilm: (2173199, 384)
Loaded mpnet: (2173199, 768)
Loaded distiluse: (2173199, 512)
Loaded e5small: (2173199, 384)
Loaded KNN (minilm): (2173199, 5)
Loaded KNN (mpnet): (2173199, 5)
Loaded KNN (distiluse): (2173199, 5)
Loaded KNN (e5small): (2173199, 5)
Product types: 13,295
Splits: train=1,847,219, val=217,319, test=108,661


In [14]:
# Create model
model = EnsembleModel(cfg, data["num_product_types"])
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

Model parameters: 4,247,105


In [15]:
# =============================================================================
# Initialize W&B Logger
# =============================================================================
wandb_logger = create_wandb_logger(cfg)
print(f"W&B run: {wandb_logger.experiment.name}")
print(f"W&B project: {cfg.wandb_project}")

# =============================================================================
# Callbacks
# =============================================================================
callbacks = [
    ModelCheckpoint(
        dirpath=cfg.output_dir,
        filename="ensemble-{epoch:02d}-{val_mape:.2f}",
        monitor="val_mape",
        mode="min",
        save_top_k=3,
    ),
    EarlyStopping(monitor="val_mape", patience=cfg.patience, mode="min", verbose=True),
    LearningRateMonitor(logging_interval="step"),
    SamplePredictionCallback(num_samples=100),
    MetricHistoryCallback(),
]

# =============================================================================
# Trainer with W&B
# =============================================================================
trainer = pl.Trainer(
    max_epochs=cfg.epochs,
    accelerator="auto",
    devices="auto",
    precision="16-mixed",
    gradient_clip_val=cfg.gradient_clip_val,
    callbacks=callbacks,
    logger=wandb_logger,  # ← W&B experiment tracking
    val_check_interval=cfg.val_check_interval,
    log_every_n_steps=cfg.log_every_n_steps,
    enable_progress_bar=True,
)

print(f"Trainer initialized with {len(callbacks)} callbacks")

wandb: WARNING The anonymous setting has no effect and will be removed in a future version.


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


W&B run: ensemble_mape_20260125_004624
W&B project: amazon-product-length
Trainer initialized with 7 callbacks


In [16]:
# Train
trainer.fit(model, train_loader, val_loader)

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:881: Checkpoint directory /kaggle/working exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/model_summary/model_summary.py:242: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.


┏━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name        ┃ Type       ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ product_emb │ Embedding  │  1.7 M │ train │     0 │
│ 1 │ knn_proj    │ Sequential │  1.7 K │ train │     0 │
│ 2 │ head        │ MLPHead    │  2.5 M │ train │     0 │
└───┴─────────────┴────────────┴────────┴───────┴───────┘

Trainable params: 4.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.2 M                                                                                                
Total estimated model params size (MB): 16                                                                         
Modules in train mode: 20                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

Metric val_mape improved. New best score: 95.180
Metric val_mape improved by 21.710 >= min_delta = 0.0. New best score: 73.469
Metric val_mape improved by 4.806 >= min_delta = 0.0. New best score: 68.663
Metric val_mape improved by 4.462 >= min_delta = 0.0. New best score: 64.201
Metric val_mape improved by 1.469 >= min_delta = 0.0. New best score: 62.732
Metric val_mape improved by 4.377 >= min_delta = 0.0. New best score: 58.355
Metric val_mape improved by 0.192 >= min_delta = 0.0. New best score: 58.163
Metric val_mape improved by 1.245 >= min_delta = 0.0. New best score: 56.917
Metric val_mape improved by 0.939 >= min_delta = 0.0. New best score: 55.978
Metric val_mape improved by 0.793 >= min_delta = 0.0. New best score: 55.186
Metric val_mape improved by 1.035 >= min_delta = 0.0. New best score: 54.151
Metric val_mape improved by 0.090 >= min_delta = 0.0. New best score: 54.060
Metric val_mape improved by 0.115 >= min_delta = 0.0. New best score: 53.945
Metric val_mape improved b

## 9. Evaluation

In [17]:
# Load best checkpoint
best_ckpt = trainer.checkpoint_callback.best_model_path
print(f"Best checkpoint: {best_ckpt}")

model = EnsembleModel.load_from_checkpoint(best_ckpt, config=cfg, num_product_types=data["num_product_types"])
model.eval()
model.cuda()

Best checkpoint: /kaggle/working/ensemble-epoch=05-val_mape=52.81.ckpt


EnsembleModel(
  (product_emb): Embedding(13295, 128, padding_idx=0)
  (knn_proj): Sequential(
    (0): Linear(in_features=20, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (head): MLPHead(
    (net): Sequential(
      (0): Linear(in_features=2208, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=1024, out_features=256, bias=True)
      (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
      (8): Linear(in_features=256, out_features=64, bias=True)
      (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): Dropout(p=0.2, inplace=False)
      (12): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)

In [18]:
@torch.no_grad()
def collect_predictions(model, dataloader, device, use_log_target: bool = False):
    """Collect predictions from a dataloader."""
    model.eval()
    preds, targets, ptypes = [], [], []
    
    for batch in tqdm(dataloader, desc="Predicting"):
        text_emb = batch["text_embedding"].to(device)
        product_type = batch["product_type"].to(device)
        knn = batch.get("knn_features")
        if knn is not None:
            knn = knn.to(device)
        
        pred = model(text_emb, product_type, knn)
        pred = F.relu(pred) + EPSILON
        
        # Transform back from log-space if needed
        if use_log_target:
            pred = torch.expm1(pred)
        
        preds.append(pred.cpu().numpy())
        if "target" in batch:
            target = batch["target"]
            if use_log_target:
                target = torch.expm1(target)  # Transform back
            targets.append(target.numpy())
        ptypes.append(batch["product_type"].numpy())
    
    result = {
        "preds": np.concatenate(preds),
        "product_types": np.concatenate(ptypes),
    }
    if targets:
        result["targets"] = np.concatenate(targets)
    return result

In [19]:
# Evaluate on validation set
device = next(model.parameters()).device
val_results = collect_predictions(model, val_loader, device, use_log_target=cfg.use_log_target)

# Create snapper from training data
snapper = create_snapper(
    data["targets"][splits["train"]],
    data["product_types"][splits["train"]],
)

# Evaluate validation set
val_raw_mape = mape_numpy(val_results["targets"], val_results["preds"])
val_raw_rmsle = rmsle_numpy(val_results["targets"], val_results["preds"])
val_snapped_preds = snapper.process(val_results["preds"], val_results["product_types"])
val_snapped_mape = mape_numpy(val_results["targets"], val_snapped_preds)
val_snapped_rmsle = rmsle_numpy(val_results["targets"], val_snapped_preds)

print(f"\n{'='*60}")
print(f"VALIDATION RESULTS (loss_fn={cfg.loss_fn}, log_target={cfg.use_log_target})")
print(f"{'='*60}")
print(f"  Raw MAPE: {val_raw_mape:.2f}%")
print(f"  Raw RMSLE: {val_raw_rmsle:.4f}")
print(f"  Snapped MAPE: {val_snapped_mape:.2f}%")
print(f"  Snapped RMSLE: {val_snapped_rmsle:.4f}")
print(f"  Improvement: {val_raw_mape - val_snapped_mape:.2f}%")

Predicting:   0%|          | 0/213 [00:00<?, ?it/s]

Snapper: 10,802 unique lengths, 12,469 product types

VALIDATION RESULTS (loss_fn=mape, log_target=True)
  Raw MAPE: 52.82%
  Raw RMSLE: 1.3694
  Snapped MAPE: 53.16%
  Snapped RMSLE: 1.3270
  Improvement: -0.34%


In [20]:
# Evaluate on test set
test_results = collect_predictions(model, test_loader, device, use_log_target=cfg.use_log_target)

test_raw_mape = mape_numpy(test_results["targets"], test_results["preds"])
test_raw_rmsle = rmsle_numpy(test_results["targets"], test_results["preds"])
test_snapped_preds = snapper.process(test_results["preds"], test_results["product_types"])
test_snapped_mape = mape_numpy(test_results["targets"], test_snapped_preds)
test_snapped_rmsle = rmsle_numpy(test_results["targets"], test_snapped_preds)

print(f"\n{'='*60}")
print(f"TEST RESULTS (loss_fn={cfg.loss_fn}, log_target={cfg.use_log_target})")
print(f"{'='*60}")
print(f"  Raw MAPE: {test_raw_mape:.2f}%")
print(f"  Raw RMSLE: {test_raw_rmsle:.4f}")
print(f"  Snapped MAPE: {test_snapped_mape:.2f}%")
print(f"  Snapped RMSLE: {test_snapped_rmsle:.4f}")
print(f"  Improvement: {test_raw_mape - test_snapped_mape:.2f}%")
print(f"\n📊 Competition Score: {max(0, 100 * (1 - test_snapped_mape / 100)):.2f}")

# =============================================================================
# Log Final Results to W&B
# =============================================================================
wandb.log({
    # Validation metrics
    "final_val_mape_raw": val_raw_mape,
    "final_val_mape_snapped": val_snapped_mape,
    "final_val_rmsle_raw": val_raw_rmsle,
    "final_val_rmsle_snapped": val_snapped_rmsle,
    # Test metrics
    "final_test_mape_raw": test_raw_mape,
    "final_test_mape_snapped": test_snapped_mape,
    "final_test_rmsle_raw": test_raw_rmsle,
    "final_test_rmsle_snapped": test_snapped_rmsle,
    # Competition score
    "final_competition_score": max(0, 100 * (1 - test_snapped_mape / 100)),
    # Post-processing improvement
    "val_snapping_improvement": val_raw_mape - val_snapped_mape,
    "test_snapping_improvement": test_raw_mape - test_snapped_mape,
})

print("\n✅ Final metrics logged to W&B")

Predicting:   0%|          | 0/107 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7bea28d97d80><function _MultiProcessingDataLoaderIter.__del__ at 0x7bea28d97d80>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
      File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
      File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
if w.is_alive():    
if w.is_alive():
       Exception ignored in:    <function _MultiProcessingDataLoaderIter.__del__ at 0x7bea28d97d80>   
 ^^^Exception ignored in: Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x


TEST RESULTS (loss_fn=mape, log_target=True)
  Raw MAPE: 50.58%
  Raw RMSLE: 1.3666
  Snapped MAPE: 51.89%
  Snapped RMSLE: 1.3248
  Improvement: -1.31%

📊 Competition Score: 48.11

✅ Final metrics logged to W&B


In [21]:
# =============================================================================
# Finish W&B Run
# =============================================================================

# Log summary metrics to W&B
wandb.summary.update({
    "best_val_mape": val_snapped_mape,
    "best_test_mape": test_snapped_mape,
    "competition_score": max(0, 100 * (1 - test_snapped_mape / 100)),
    "best_checkpoint": best_ckpt,
})

# Finish the W&B run
wandb.finish()
print("✅ W&B run completed and synced!")

epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇████████
final_competition_score,▁
final_test_mape_raw,▁
final_test_mape_snapped,▁
final_test_rmsle_raw,▁
final_test_rmsle_snapped,▁
final_val_mape_raw,▁
final_val_mape_snapped,▁
final_val_rmsle_raw,▁
final_val_rmsle_snapped,▁
+16,...


✅ W&B run completed and synced!


## 10. Analysis (Optional)

Run the next cell to analyze prediction errors by product type.

In [22]:
# Optional: Analyze errors by product type
def analyze_errors_by_type(preds: np.ndarray, targets: np.ndarray, product_types: np.ndarray, top_k: int = 20):
    """Analyze MAPE by product type to identify problem areas."""
    unique_types = np.unique(product_types)
    type_errors = []
    
    for ptype in unique_types:
        mask = product_types == ptype
        if mask.sum() < 10:  # Skip rare types
            continue
        type_mape = mape_numpy(targets[mask], preds[mask])
        type_errors.append({
            "product_type": int(ptype),
            "count": int(mask.sum()),
            "mape": type_mape,
            "mean_target": float(targets[mask].mean()),
            "std_target": float(targets[mask].std()),
        })
    
    df = pd.DataFrame(type_errors).sort_values("mape", ascending=False)
    print(f"\n📊 Top {top_k} Worst Product Types by MAPE:")
    print(df.head(top_k).to_string(index=False))
    
    print(f"\n📊 Top {top_k} Best Product Types by MAPE:")
    print(df.tail(top_k).to_string(index=False))
    
    return df

# Uncomment to run analysis:
error_analysis = analyze_errors_by_type(test_snapped_preds, test_results["targets"], test_results["product_types"])


📊 Top 20 Worst Product Types by MAPE:
 product_type  count        mape  mean_target  std_target
        12843     16 4936.958984   782.178650  511.498199
         7244     10 2425.075439   517.517334  335.068817
          727     12 1180.388672   661.578125  355.773560
        13170     45  890.932739   755.858826  599.768677
         2521     21  622.372559  1200.628784  607.809082
         2948     22  511.731659   706.349304  368.515106
         2275    163  508.802612   942.381409  772.823853
         3298    180  486.257324   901.300354  328.622162
        11872     22  381.994141   482.800079  329.424988
        13218     16  359.625732   590.777588  523.759338
         5082     20  346.414124  1096.650024  512.888184
         3432     10  300.478302   325.284973  234.886520
         6501     16  280.408752   665.902588  360.324066
         7789     10  236.194443   706.709290  334.541534
          142     27  221.893982   696.344238  193.308929
         1148     12  218.250229 

## 📝 Experiment Log

Track your experiments here:

| Experiment | Loss | Log Target | Val MAPE | Test MAPE | Notes |
|------------|------|------------|----------|-----------|-------|
| Baseline | mape | False | 51.78% | - | Current best |
| Exp 2 | rmsle | False | - | - | |
| Exp 3 | log_mape | False | - | - | |
| Exp 4 | focal_mape | False | - | - | γ=2.0 |
| Exp 5 | mape | True | - | - | Log-transform target |
| Exp 6 | combined | False | - | - | 0.7·MAPE + 0.3·RMSLE |

### Next Steps After Loss Experiments
1. **Ensemble**: Train top 3 losses, average predictions
2. **Architecture**: Try larger KNN projection (32 → 64)
3. **Learning Rate**: Try lr=3e-4 or lr=5e-4
4. **Deeper MLP**: [1024, 512, 256, 64] instead of [1024, 256, 64]